# Mental health counseling ChatGPT Clone from Scratch

## Task 1: Import Libraries

In [1]:
# import the librarys here
import os
from datasets import load_dataset
import json
from enum import Enum
import random
from openai import OpenAI

## Task 2 : Create Classes for Dataset

In [2]:
#  define the RoleType Enum here
class RoleType(Enum):
    USER = 'user'
    SYSTEM = 'system'
    ASSISTANT = 'assistant'


In [3]:
# define the Role class here
class Role:
    def __init__(self, role_type: RoleType, content):
        self.role = role_type.value
        self.content = content
        self.value = {'role': self.role, 'content': self.content}


In [4]:
#  define the messsage class here
class Message:
    def __init__(self, user_content, system_content, assistant_content):
        self.user_role = Role(role_type=RoleType.USER, content=user_content)
        self.system_role = Role(role_type=RoleType.SYSTEM, content=system_content)
        self.assistant_role = Role(role_type=RoleType.ASSISTANT, content=assistant_content)
        self.message = {'messages':[self.system_role.value, self.user_role.value, self.assistant_role.value, ]} 


## Task 3 : Load, Explore and Store the Data

In [5]:
# load the dataset 
dataset = load_dataset("Amod/mental_health_counseling_conversations", split='train')

/Users/jyotishkumar/opt/anaconda3/lib/python3.8/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [6]:
# Sample Message object
context = dataset[152]['Context']
response = dataset[152]['Response']
system_content = "You serve as a supportive and honest psychology and psychotherapy assistant. Your main duty is to offer compassionate, understanding, and non-judgmental responses to users seeking emotional and psychological assistance. Respond with empathy and exhibit active listening skills. Your replies should convey that you comprehend the user’s emotions and worries. In cases where a user mentions thoughts of self-harm, suicide, or harm to others, prioritize their safety. Encourage them to seek immediate professional help and provide emergency contact details as needed. It’s important to note that you are not a licensed medical professional. Refrain from diagnosing or prescribing treatments. Instead, guide users to consult with a licensed therapist or medical expert for tailored advice. Never store or disclose any personal information shared by users. Uphold their privacy at all times. Avoid taking sides or expressing personal viewpoints. Your responsibility is to create a secure space for users to express themselves and reflect. Always aim to foster a supportive and understanding environment for users to share their emotions and concerns. Above all, prioritize their well-being and safety."
message_obj = Message(user_content=context, system_content=system_content, assistant_content = response )

print(message_obj.message)

{'messages': [{'role': 'system', 'content': 'You serve as a supportive and honest psychology and psychotherapy assistant. Your main duty is to offer compassionate, understanding, and non-judgmental responses to users seeking emotional and psychological assistance. Respond with empathy and exhibit active listening skills. Your replies should convey that you comprehend the user’s emotions and worries. In cases where a user mentions thoughts of self-harm, suicide, or harm to others, prioritize their safety. Encourage them to seek immediate professional help and provide emergency contact details as needed. It’s important to note that you are not a licensed medical professional. Refrain from diagnosing or prescribing treatments. Instead, guide users to consult with a licensed therapist or medical expert for tailored advice. Never store or disclose any personal information shared by users. Uphold their privacy at all times. Avoid taking sides or expressing personal viewpoints. Your responsib

In [7]:
#  create the train_dataset variable
sampled_dataset = random.choices(dataset, k=100)
train_dataset = []

# Print the sampled data to verify
print(sampled_dataset[1])

for row in sampled_dataset:
    message_obj = Message(user_content=row['Context'], system_content=system_content, assistant_content=row['Response'])
    train_dataset.append(message_obj.message)

print(train_dataset[1])

{'Context': "I have a fear of something and I want to face that fear to overcome it, but I don't know how. What can I do?", 'Response': 'Biologically, fear is designed to protect us from harm. Fear is not always a bad thing, and in fact can be quite healthy and appropriate depending on the situation.\xa0A phobia, however, is different. You used the term "fear" rather than "phobia". A phobia is an irrational fear - meaning it is not rational for you to fear that thing. If your situation is a phobia, exposure therapy can help. This consists of gradually exposing yourself a little at a time to the thing you are afraid of. Some people with phobias find that the irrational fear interferes with their life and they do need to overcome it. Someone who is afraid to drive over bridges may go to great lengths to avoid routes that have bridges. People who are afraid of elevators may always use the stairs instead, which may not always be feasible. If overcoming a phobia will improve the quality of 

In [11]:
# save data in JSONl format 
def save_to_jsonl(data, file_path):
    with open(file_path, 'w') as file:
        for row in data:
            line = json.dumps(row)
            file.write(line + '\n')


# Store the data in JSONL format
training_data_path = '/Users/jyotishkumar/PROJECTS/jyotish/PYTHON/chatbot_ai/train.jsonl'
save_to_jsonl(train_dataset[:-5], training_data_path)

validation_data_path = '/Users/jyotishkumar/PROJECTS/jyotish/PYTHON/chatbot_ai/validation.jsonl'
save_to_jsonl(train_dataset[-5:], validation_data_path)

## Task 4 : Fine Tune the Model

In [12]:
# load the training and validation files
training_data = open(training_data_path, "rb")
validation_data = open(validation_data_path, "rb")

In [16]:
#  add OpenAI api_key
api_key = 'YOUR-API=KEY'
client = OpenAI(api_key=api_key)

In [17]:
# Upload the training and validation files
training_response = client.files.create(file=training_data, purpose="fine-tune")
training_file_id = training_response.id

validation_response = client.files.create(file=validation_data, purpose="fine-tune")
validation_file_id = validation_response.id

print("Training file id:", training_file_id)
print("Validation file id:", validation_file_id)

Training file id: file-h5uyQLvvsElkHIuBVxeMKEF7
Validation file id: file-Q6CTNqW8yAkcBBN2rss0bG3V


In [18]:
# create a fine-tuning job
response = client.fine_tuning.jobs.create(
    training_file=training_file_id,
    model="gpt-3.5-turbo",
    suffix="my-test-model",
    validation_file=validation_file_id
)

job_id = response.id

print(response)

FineTuningJob(id='ftjob-wEcKcvJQBV4pH6PEGl1PvV69', created_at=1731824905, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-c2jI2lYztN2xjHAYTUiSmsVv', result_files=[], seed=1615570161, status='validating_files', trained_tokens=None, training_file='file-h5uyQLvvsElkHIuBVxeMKEF7', validation_file='file-Q6CTNqW8yAkcBBN2rss0bG3V', estimated_finish=None, integrations=[], user_provided_suffix='my-test-model')


In [19]:
# retrieve the job status
job_id = response.id

job_status = client.fine_tuning.jobs.retrieve(job_id)
print(job_status)

FineTuningJob(id='ftjob-wEcKcvJQBV4pH6PEGl1PvV69', created_at=1731824905, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-c2jI2lYztN2xjHAYTUiSmsVv', result_files=[], seed=1615570161, status='running', trained_tokens=None, training_file='file-h5uyQLvvsElkHIuBVxeMKEF7', validation_file='file-Q6CTNqW8yAkcBBN2rss0bG3V', estimated_finish=1731825491, integrations=[], user_provided_suffix='my-test-model')


## Task 5 : Test the Fine-Tuned Model

In [20]:
# create and store message dictionaries
system_message = """You serve as a supportive and honest psychology and psychotherapy assistant. Your main duty is to offer compassionate, understanding, and non-judgmental responses to users seeking emotional and psychological assistance. Respond with empathy and exhibit active listening skills. Your replies should convey that you comprehend the user's emotions and worries. In cases where a user mentions thoughts of self-harm, suicide, or harm to others, prioritize their safety. Encourage them to seek immediate professional help and provide emergency contact details as needed. It's important to note that you are not a licensed medical professional. Refrain from diagnosing or prescribing treatments. Instead, guide users to consult with a licensed therapist or medical expert for tailored advice. Never store or disclose any personal information shared by users. Uphold their privacy at all times. Avoid taking sides or expressing personal viewpoints. Your responsibility is to create a secure space for users to express themselves and reflect. Always aim to foster a supportive and understanding environment for users to share their emotions and concerns. Above all, prioritize their well-being and safety."""

messages = []
messages.append({"role": "system", "content": system_message})
user_message = "Every winter I find myself getting sad because of the weather. How can I fight this?"
messages.append({"role": "user", "content": user_message})

In [21]:
# test the fine-tuned chat completion model
completion = client.chat.completions.create(
    model= response.model,
    messages=messages
)
print(completion.choices[0].message)

ChatCompletionMessage(content="I'm sorry to hear that you're feeling down during the winter months. It's not uncommon to experience seasonal changes in mood, known as Seasonal Affective Disorder (SAD). Here are some strategies that may help you combat these feelings:\n\n1. **Light Therapy**: Consider using a light therapy box that mimics natural sunlight to help regulate your mood.\n2. **Regular Exercise**: Physical activity can boost your mood and energy levels. Even a short walk outside can make a difference.\n3. **Healthy Diet**: Eating nutritious foods can support your mental health. Consider adding more fruits, vegetables, and whole grains to your meals.\n4. **Social Connections**: Stay connected with loved ones and friends, even if it's through virtual means. Social support can be a great source of comfort.\n5. **Mindfulness and Relaxation Techniques**: Practices like meditation, deep breathing, or yoga can help reduce stress and improve your mood.\n6. **Professional Help**: Cons

In [23]:
# Get and compare the output of the gpt-3.5-turbo chat completion model
completion = client.chat.completions.create(
    model= "gpt-3.5-turbo",
    messages=messages
)
print(completion.choices[0].message)

ChatCompletionMessage(content="It's common to feel a bit down during the winter months due to the changes in weather and light. One approach to combatting this could be trying to maximize exposure to natural light by spending time outdoors during daylight hours or using a light therapy box.\n\nEngaging in regular physical activity, eating a balanced diet, and maintaining a regular sleep schedule can also help boost your mood. Additionally, staying socially connected with friends and loved ones, as well as practicing relaxation techniques such as deep breathing or mindfulness, may also contribute to improving your overall well-being.\n\nIf you find that your winter blues impact your daily functioning or persist despite these efforts, it may be beneficial to consult with a mental health professional for further support and guidance. Remember, you are not alone in feeling this way, and seeking help is a positive step towards feeling better.", refusal=None, role='assistant', audio=None, fu